In [1]:
#@title ✅ Step 1 – Install dependencies
!pip -q install fastapi "pydantic<2" uvicorn streamlit requests pyngrok
import os, time, pathlib, subprocess, sys, json, requests


In [2]:
#@title 🔑 Register your ngrok authtoken (only once per runtime)
!ngrok config add-authtoken 33ifhOJyJaahIBESM3xGUc7rIE3_3nYCncpMXCQpMQWySisu1


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
#@title ✅ Step 3 – Write FastAPI backend (main.py)
backend_dir = pathlib.Path("backend")
backend_dir.mkdir(exist_ok=True, parents=True)

fastapi_code = r'''
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import time

app = FastAPI(title="FinSynth Backend — Track B")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"]
)

class Query(BaseModel):
    text: str

@app.get("/health")
def health():
    return {"status": "ok", "timestamp": time.time()}

@app.post("/qa")
def qa(q: Query):
    user_q = q.text.lower()
    if "market" in user_q:
        answer = "📈 AI stock markets show positive momentum driven by strong sentiment and LLM growth signals."
        citations = ["FinDoc23:L120-L140", "NewsAI12:L5-L18"]
    elif "energy" in user_q:
        answer = "⚡ Energy sector is stable; WTI prices are showing balanced supply-demand patterns."
        citations = ["EnergyReport_15:L40-L65", "MarketBrief_R8:L12-L22"]
    elif "ev" in user_q or "factory" in user_q:
        answer = "🚗 EV manufacturing efficiency rose 12 % in Q2 due to AI-driven automation."
        citations = ["AutoInsightQ2:L30-L42"]
    else:
        answer = f"🤖 Demo response for: {q.text}"

    trace = [
        {"hop": 1, "subq": "Entity recognition and graph lookup"},
        {"hop": 2, "subq": "Neighbor expansion and evidence retrieval"},
        {"hop": 3, "subq": "Answer composition and reasoning"}
    ]
    latency = round(time.time() % 10 + 0.3, 2)
    return {"answer": answer, "citations": citations, "trace": trace, "latency": latency}
'''
with open(backend_dir / "main.py", "w") as f:
    f.write(fastapi_code)
print("✅ backend/main.py written")


✅ backend/main.py written


In [4]:
#@title ✅ Step 4 – Start FastAPI backend (port 8000)
proc = subprocess.Popen(
    [sys.executable, "-m", "uvicorn", "backend.main:app",
     "--host", "0.0.0.0", "--port", "8000"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(3)
print("Health:", requests.get("http://127.0.0.1:8000/health").json())


Health: {'status': 'ok', 'timestamp': 1760405561.4266605}


In [5]:
#@title ✅ Step 5 – Write Streamlit frontend (app.py)
app_dir = pathlib.Path("app")
app_dir.mkdir(exist_ok=True, parents=True)

streamlit_code = r'''
import os, time, requests, streamlit as st

st.set_page_config(page_title="FinSynth — Week 7", layout="wide")

# ✅ Direct backend URL (no secrets.toml needed)
API_URL = os.environ.get("API_URL", "http://localhost:8000")

st.title("💹 FinSynth — Multi-Agent Financial RAG App")
st.caption("A Graph-RAG + Multi-Hop Reasoning demo (Track B Deployment)")
st.markdown("---")

query = st.text_area("🔍 Enter your question:", height=120,
                    placeholder="e.g. What is the latest AI market trend in energy sector?")

if st.button("🚀 Ask Model"):
    if not query.strip():
        st.warning("Please enter a question first.")
    else:
        t0 = time.time()
        try:
            response = requests.post(f"{API_URL}/qa", json={"text": query}, timeout=60)
            data = response.json()
            latency = time.time() - t0

            st.subheader("🧠 Answer")
            st.write(data.get("answer", "(no answer)"))
            st.caption(f"⏱️ Latency: {latency:.2f}s")

            st.markdown("### 📚 Citations")
            for c in data.get("citations", []):
                st.write("• ", c)

            st.markdown("### 🔁 Reasoning Trace")
            for hop in data.get("trace", []):
                st.write(f"Hop {hop['hop']}: {hop['subq']}")
        except Exception as e:
            st.error(f"❌ Error: {e}")

st.sidebar.title("ℹ️ App Info")
st.sidebar.markdown("**Author:** Hariram Sabari Kriesh (UMKC)")
st.sidebar.markdown("**Project:** FinSynth — Multi-Agent GenAI Framework")
st.sidebar.markdown("**Track B:** FastAPI + Streamlit Deployment")
'''
with open(app_dir / "app.py", "w") as f:
  f.write(streamlit_code)
print("✅ app/app.py written")


✅ app/app.py written


In [6]:
#@title 🌐 Step 6 – Launch Streamlit App (via ngrok public URL)
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🚀 Streamlit App URL:", public_url.public_url)

!streamlit run app/app.py --server.port 8501 > /dev/null 2>&1 &
time.sleep(5)
print("✅ Streamlit is running! Click the URL above.")


🚀 Streamlit App URL: https://citizenly-hypocoristic-crissy.ngrok-free.dev
✅ Streamlit is running! Click the URL above.


In [7]:
!pkill -f uvicorn || echo "No backend process to kill"
!python3 -m uvicorn backend.main:app --host 0.0.0.0 --port 8000 > /dev/null 2>&1 &
import time, requests
time.sleep(4)
print("Health:", requests.get("http://127.0.0.1:8000/health").json())


^C
Health: {'status': 'ok', 'timestamp': 1760405571.324912}


In [8]:
API_URL = os.environ.get("API_URL", "http://localhost:8000")


In [9]:
from pyngrok import ngrok
import time

public_url = ngrok.connect(8501)
print("🚀 Streamlit App URL:", public_url.public_url)

!streamlit run app/app.py --server.port 8501 > /dev/null 2>&1 &
time.sleep(5)
print("✅ Streamlit is now running. Click the link above!")


🚀 Streamlit App URL: https://citizenly-hypocoristic-crissy.ngrok-free.dev
✅ Streamlit is now running. Click the link above!


In [10]:
#@title ✅ Step 7 – Evaluation Logger (safe JSON version)
import pandas as pd, time, requests

queries = [
  "Explain AI stock momentum this month.",
  "Summarize recent energy market stability.",
  "How did automation affect EV production?",
  "Which factors influence tech stock growth?",
  "Compare WTI and Nasdaq trends."
]

results = []
for q in queries:
    t0 = time.time()
    try:
        r = requests.post("http://127.0.0.1:8000/qa", json={"text": q}, timeout=20)
        try:
            data = r.json()
        except Exception:
            print("❌ Non-JSON response for:", q)
            print("Raw output:", r.text[:200])
            continue
        latency = round(time.time() - t0, 2)
        results.append({
            "query": q,
            "latency": latency,
            "answer": data.get("answer", "(no answer)")
        })
        print(f"✅ {q[:40]}...  ({latency}s)")
    except Exception as e:
        print(f"⚠️ Request failed for: {q[:40]} — {e}")

df = pd.DataFrame(results)
df.to_csv("evaluation_log.csv", index=False)
print("\n✅ Logged", len(df), "successful queries to evaluation_log.csv")
df


❌ Non-JSON response for: Explain AI stock momentum this month.
Raw output: Internal Server Error
✅ Summarize recent energy market stability...  (0.01s)
✅ How did automation affect EV production?...  (0.0s)
❌ Non-JSON response for: Which factors influence tech stock growth?
Raw output: Internal Server Error
❌ Non-JSON response for: Compare WTI and Nasdaq trends.
Raw output: Internal Server Error

✅ Logged 2 successful queries to evaluation_log.csv


,query,latency,answer
0,Summarize recent energy market stability.,0.01,📈 AI stock markets show positive momentum driv...
1,How did automation affect EV production?,0.00,🚗 EV manufacturing efficiency rose 12 % in Q2 ...
